# Aim
This script shows how to estimate spillover from single metal spots on an agarose coated slide.
Each spot should be imaged with a single acquisition. The name of the acquisition should be the metal that is used:
E.g. PanormaA_1_Yb176_23.txt

When run with the example data it reproduces the spillover estimation shown in Fig S5A as well as Fig 4A

# Script
## load all libraries

In [2]:
library(CATALYST)
library(data.table)
library(ggplot2)
library(flowCore)
library(dplyr)
library(dtplyr)
library(stringr)
library(ggpmisc)

In [3]:
source('workflow/scripts/utils_compensation/spillover_imc_helpers.R')

## setup the configuration variables

In [4]:
# list of folders that contain each a complete single stain acquisition (e.g. in case that one wants to run and compare multiple single stains from different days)
fols_ss = snakemake@input$fns_compensation

# needsto contain all the channels used for the single stains
fns_pannel = c(snakemake@input$fn_ss_panel)

# output folder
fol_out = '../../figures/'
# name prefix for all output
prefix ='spill_'

In [5]:
dat_pannel = rbindlist(lapply(fns_pannel, function(fn) fread(fn, header = F)), fill=T)
setnames(dat_pannel, 'metal')
                              
ss_mass = unique(dat_pannel[, list(mass=as.numeric(str_extract_all(.BY[[1]], "[0-9]+")[[1]])), metal]$mass)

## load single stains
### Data loading

In [6]:
# load the data
list_img_ss <-lapply(fols_ss, load_ss_zip)
#list_img_ss = rbindlist(list_img_ss)
names(list_img_ss) <- fols_ss[1]

### Adapt the column names to be recognized metal names by CATALYST 
CATALYST needs to have the metal names in the format (METAL)(MASS)Di

In [7]:
list_img_ss = lapply(list_img_ss, function(x) lapply(x, fixnames))
dats_raw = lapply(list_img_ss, imglist2dat)

# this makes this script ignore the fact that the singlestains come from different acquisitions.
dats_raw = list('singlestains'=rbindlist(dats_raw))

### Extract the single stain masses from the acquisition name
This needs to be changed in case a different naming scheme is used!

In [8]:
for (dat in dats_raw){
  dat[, metal:= strsplit(.BY[[1]], '_')[[1]][3],by=file]
  dat[, mass:= as.numeric(str_extract_all(.BY[[1]], "[0-9]+")[[1]]),by=metal]
}

## Visualization of the raw data
In the following section the raw data is visualized

### Calculate per-file medians

In [9]:
dats_raw_sum = rbindlist(lapply(dats_raw, calc_file_medians),idcol = T)

### Visualize per-file medians
Plots the median of the data. It is recommended to have >200 counts for all the channels.
This is also a good plot to check if the metal spots really contain the correct metal!

In [10]:
dats_raw_sum %>%
  ggplot(aes(x=1, y=med, color=.id))+
  facet_wrap(~file+metal, scales = 'free_y')+
  geom_label(aes(label=variable), size=4)

###  Optional data bining

If the median per-pixel intensities are to low, it could be worth to sum up some consecuteive pixels to get a better accuracy for the estimation
(here not the case). This is valid because for segmentation based quantitative image analysis usually anyways pixels are aggregated. If the binning is choosen to big, there is however a potential accumulation of background noise.

In [11]:
# defines over how many pixels the aggregation should happen
# 1 = no aggregation
npixelbin = 10

dats_agg <- lapply(dats_raw, function(x) aggregate_pixels(x, n=npixelbin))
dats_agg_sum = rbindlist(lapply(dats_agg, calc_file_medians), idcol = T)

### Visualize per-file medians after binning
The intensities increase according to the aggregation factor

In [12]:
dats_agg_sum %>%
  ggplot(aes(x=1, y=med, color=.id))+
  facet_wrap(~file+metal, scales = 'free_y', ncol=3)+
  geom_label(aes(label=variable))


## CATALYST based compensation

## estimate the spillover
To estimate the spillover, the (aggregated) pixel values are first debarcoded using CATALYST, treating them like single cells. This step acts as a quality filter to remove background/noisy/weak pixels as well as pixels with artefacts (e.g. specles with strong signal in many channels).
If the true metal was correctly encoded in the filename, the 'remove_incorrect_bc' option will check the debarcoding and remove events assigned to the wrong barcode.

Then this identified, strong single stain pixels will be used for the spillover estimation.

In [13]:
res = lapply(dats_agg, function(x) re_from_dat(x,
                                               ss_ms=ss_mass,
                                               minevents = 40,
                                              correct_bc = F))
sms = lapply(res, function(x) computeSpillmat(x, method='classic'))



In [14]:
sm = sms[[1]]

### save the spillover matrix

In [15]:
write.csv(sm,file = snakemake@output[[1]])

### Visualization of the spillover matrix

### Some quality indicators

Here we calculate e.g. number of debarcoded events/metal, median levels of highest signal and second highest signal after subtracting background

In [16]:

for (i in seq_along(res)){
  
  dat = dats_agg[[i]]
  re = res[[i]]
  
  name = names(dats_agg)[i]
  tdat = dat %>%
    mutate(bcid = bc_ids(re)) %>%
    filter(bcid != '0') %>%
    dplyr::select(-c(Start_push, End_push, Pushes_duration,   X , Y  ,Z)) %>%
  melt.data.table(id.vars = c('metal', 'mass','file', 'bcid')) %>%
  do(data.table(.)[, list(med=median(value), n=.N), by=.(variable, metal, mass, bcid,file)]) 
  
  tdat[, overall_med:= median(med), by='variable']
  tdat[, med:=med-overall_med]
  # find the highest metal, second highest metal after subtracting background (=overal median)
  sumdat = tdat[ , .(
    highestvariable = variable[med == max(med)],
    highestmed = max(med),
    secondhighestvariable = variable[med == sort(med,partial=length(med)-1)[length(med)-1]],
    secondhighestmed = sort(med,partial=length(med)-1)[length(med)-1],
    thirdhighestvariable = variable[med == sort(med,partial=length(med)-2)[length(med)-2]],
    thirdhighestmed = sort(med,partial=length(med)-2)[length(med)-2],
    n=max(n)
  )  ,by=.( mass, bcid,file)]
  
  print(sumdat)
}


In [17]:
sessionInfo()